In [1]:
from utils import TokenQuestion, Word2VecSolver, RetrievalSolver, Question, Solver
import collections
import jieba
import pandas as pd
import numpy as np
import subprocess
from subprocess import PIPE
import shlex
import re
import csv
from time import time
jieba.set_dictionary('data/DICT_CK+jieba_lower')
jieba.add_word('龐燮傍謝', freq=10, tag='xx')
ANS = ['a', 'b', 'c', 'd', 'e']
WINDOW = 10
VEC_SIZE = 100


Building prefix dict from /Users/tripper/Desktop/project/pixnet-nlp/data/DICT_CK+jieba_lower ...
Loading model from cache /var/folders/4_/b_9vfbwx41g4t48jrzr0r4vm0000gn/T/jieba.uaab2b61a375629a86d8bcbcdd9e861b1.cache
Loading model cost 1.261 seconds.
Prefix dict has been built succesfully.


In [5]:
grep = RetrievalSolver('data/content-zh-lower.txt')

In [2]:
w2v_jieba_cbow = Word2VecSolver('w2v-experiment/model/', 'cbow')

In [3]:
val_set = pd.read_csv('question_sample/pre/raw_samples.csv')
test_set = pd.read_csv('question_sample/official/hackathon_1000.tsv', sep='\t', names=['no','content', 'ans', 'a', 'b', 'c', 'd', 'e', 'ans_ref', 'url', 'level'], quoting=csv.QUOTE_NONE)

In [14]:
# grep + word2vec(jieba+cbow) aggregation
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    grep.solve(q)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        elif len(grep.key) > 0:
            w2v_jieba_cbow.solve(q, key = grep.key)
            if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    grep.solve(q)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        elif len(grep.key) > 0:
            w2v_jieba_cbow.solve(q, key = grep.key)
            if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.77%
# total validation samples: 546
# test accuracy: 86.40%
# total test samples: 1000
# elapsed time: 508.40 sec
# rate: 0.33 sec/sample

validation accuracy: 93.77%
total validation samples: 546
test accuracy: 86.40%
total test samples: 1000
elapsed time: 508.40 sec
rate: 0.33 sec/sample


In [4]:
# direct word2vec(jieba+cbow)
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    w2v_jieba_cbow.solve(q)
    if w2v_jieba_cbow.prediction != Solver.UNSOLVED:
        if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))

temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = TokenQuestion(row, 'jieba')
    w2v_jieba_cbow.solve(q)
    if w2v_jieba_cbow.prediction != Solver.UNSOLVED:
        if w2v_jieba_cbow.prediction == q.ans: correct += 1
        total += 1

elapsed = time() - t

print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 77.47%
# total validation samples: 546
# test accuracy: 77.60%
# total test samples: 1000
# elapsed time: 2.74 sec
# rate: 0.00 sec/sample

validation accuracy: 77.47%
total validation samples: 546
test accuracy: 77.60%
total test samples: 1000
elapsed time: 2.74 sec
rate: 0.00 sec/sample


In [4]:
# grep retrieval engine
t = time()
correct, total = 0, 0
for index, row in val_set.iterrows():
    q = Question(row)
    grep.solve(q)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        total += 1
print('validation accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total validation samples: {}'.format(total))
temp = total
correct, total = 0, 0
for index, row in test_set.iterrows():
    q = Question(row)
    grep.solve(q)
    if grep.prediction != Solver.UNSOLVED:
        if grep.prediction == q.ans: correct += 1
        total += 1
elapsed = time() - t
print('test accuracy: {:.2f}%'.format(100*float(correct)/float(total)))
print('total test samples: {}'.format(total))
print('elapsed time: {:.2f} sec'.format(elapsed))
print('rate: {:.2f} sec/sample'.format(float(elapsed) / float(temp + total)))

# validation accuracy: 93.77%
# total validation samples: 546
# test accuracy: 71.40%
# total test samples: 1000
# elapsed time: 505.71 sec
# rate: 0.33 sec/sample

validation accuracy: 93.77%
total validation samples: 546
test accuracy: 71.40%
total test samples: 1000
elapsed time: 505.71 sec
rate: 0.33 sec/sample
